<a href="https://colab.research.google.com/github/DavidSenseman/BIO1173_Fall2025/blob/main/F25_Class_04_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---------------------------
**COPYRIGHT NOTICE:** This Jupyterlab Notebook is a Derivative work of [Jeff Heaton](https://github.com/jeffheaton) licensed under the Apache License, Version 2.0 (the "License"); You may not use this file except in compliance with the License. You may obtain a copy of the License at

> [http://www.apache.org/licenses/LICENSE-2.0](http://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

------------------------

# **BIO 1173: Intro Computational Biology**

##### **Module 4: ChatGPT and Large Language Models**

* Instructor: [David Senseman](mailto:David.Senseman@utsa.edu), [Department of Biology, Health and the Environment](https://sciences.utsa.edu/bhe/), [UTSA](https://www.utsa.edu/)

### Module 4 Material

* **Part 4.1: Introduction to Transformers and Accessing ChatGTP**
* Part 4.2: LLM Memory. Embedding and Prompt Engineering
* Part 4.3: Generative AI
* Part 4.4: Text to Images with Stable Diffusion


## Google CoLab Instructions

You MUST run the following code cell to get credit for this class lesson. By running this code cell, you will map your GDrive to /content/drive and print out your Google GMAIL address. Your Instructor will use your GMAIL address to verify the author of this class lesson.

In [1]:
# You must run this cell first
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    from google.colab import auth
    auth.authenticate_user()
    COLAB = True
    print("Note: Using Google CoLab")
    import requests
    gcloud_token = !gcloud auth print-access-token
    gcloud_tokeninfo = requests.get('https://www.googleapis.com/oauth2/v3/tokeninfo?access_token=' + gcloud_token[0]).json()
    print(gcloud_tokeninfo['email'])
except:
    print("**WARNING**: Your GMAIL address was **not** printed in the output below.")
    print("**WARNING**: You will NOT receive credit for this lesson.")
    COLAB = False

Mounted at /content/drive
Note: Using Google CoLab
david.senseman@gmail.com


Make sure your GMAIL address is included as the last line in the output above.

### Install Custom Functions

Run the cell below to load custom functions used in this lesson.

In [2]:
# Simple function to print out elasped time
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

# **Introduction to Transformers**

Transformers are neural networks that provide state-of-the-art solutions for many of the problems previously assigned to recurrent neural networks. [[Cite:vaswani2017attention]](https://arxiv.org/abs/1706.03762) Sequences can form both the input and the output of a neural network, examples of such configurations include::

* Vector to Sequence - Image captioning
* Sequence to Vector - Sentiment analysis
* Sequence to Sequence - Language translation

Sequence-to-sequence allows an input sequence to produce an output sequence based on an input sequence. Transformers focus primarily on this sequence-to-sequence configuration.

## High-Level Overview of Transformers

This course focuses primarily on the application of deep neural networks. The focus will be on presenting data to a transformer and a transformer's major components. As a result, we will not focus on implementing a transformer at the lowest level. The following section provides an overview of critical internal parts of a transformer, such as residual connections and attention. In the next chapter, we will use transformers from [Hugging Face](https://huggingface.co/) to perform natural language processing with transformers. If you are interested in implementing a transformer from scratch, Keras provides a comprehensive [example](https://www.tensorflow.org/text/tutorials/transformer).

Figure 10.TRANS-1 presents a high-level view of a transformer for language translation.

**Figure 10.TRANS-1: High Level View of a Translation Transformer**
![Transformer](https://data.heatonresearch.com/images/jupyter/transformer-1.jpg)

We use a transformer that translates between English and Spanish for this example. We present the English sentence "the cat likes milk" and receive a Spanish translation of "al gato le gusta la leche."

We begin by placing the English source sentence between the beginning and ending tokens. This input can be of any length, and we presented it to the neural network as a ragged Tensor. Because the Tensor is ragged, no padding is necessary. Such input is acceptable for the attention layer that will receive the source sentence. The encoder transforms this ragged input into a hidden state containing a series of key-value pairs representing the knowledge in the source sentence. The encoder understands to read English and convert to a hidden state. The decoder understands how to output Spanish from this hidden state.

We initially present the decoder with the hidden state and the starting token. The decoder will predict the probabilities of all words in its vocabulary. The word with the highest probability is the first word of the sentence.

The highest probability word is attached concatenated to the translated sentence, initially containing only the beginning token. This process continues, growing the translated sentence in each iteration until the decoder predicts the ending token.


## **Transformer Hyperparameters**

Before we describe how these layers fit together, we must consider the following transformer hyperparameters, along with default settings from the Keras transformer example:

* num_layers = 4
* d_model = 128
* dff = 512
* num_heads = 8
* dropout_rate = 0.1

Multiple encoder and decoder layers can be present. The **num_layers** hyperparameter specifies how many encoder and decoder layers there are. The expected tensor shape for the input to the encoder layer is the same as the output produced; as a result, you can easily stack these layers.

We will see embedding layers in the next chapter. However, you can think of an embedding layer as a dictionary for now. Each entry in the embedding corresponds to each word in a fixed-size vocabulary. Similar words should have similar vectors. The **d_model** hyperparameter specifies the size of the embedding vector. Though you will sometimes preload embeddings from a project such as [Word2vec](https://radimrehurek.com/gensim/models/word2vec.html) or [GloVe](https://nlp.stanford.edu/projects/glove/), the optimizer can train these embeddings with the rest of the transformer. Training your embeddings allows the **d_model** hyperparameter to set to any desired value. If you transfer the embeddings, you must set the **d_model** hyperparameter to the same value as the transferred embeddings.

The **dff** hyperparameter specifies the size of the dense feedforward layers. The **num_heads** hyperparameter sets the number of attention layers heads. Finally, the dropout_rate specifies a dropout percentage to combat overfitting. We discussed dropout previously in this book.

## **Inside a Transformer**

In this section, we will examine the internals of a transformer so that you become familiar with essential concepts such as:

* Embeddings
* Positional Encoding
* Attention and Self-Attention
* Residual Connection

You can see a lower-level diagram of a transformer in Figure 10.TRANS-2.

**Figure 10.TRANS-2: Architectural Diagram from the Paper**
![Attention is All you Need](https://data.heatonresearch.com/images/jupyter/transformer-2.jpg)

While the original transformer paper is titled "Attention is All you Need," attention isn't the only layer type you need. The transformer also contains dense layers. However, the title "Attention and Dense Layers are All You Need" isn't as catchy.

The transformer begins by tokenizing the input English sentence. Tokens may or may not be words. Generally, familiar parts of words are tokenized and become building blocks of longer words. This tokenization allows common suffixes and prefixes to be understood independently of their stem word. Each token becomes a numeric index that the transformer uses to look up the vector. There are several special tokens:

* Index 0 = Pad
* Index 1 = Unknow
* Index 2 = Start token
* Index 3 = End token

The transformer uses index 0 when we must pad unused space at the end of a tensor. Index 1 is for unknown words. The starting and ending tokens are provided by indexes 2 and 3.

The token vectors are simply the inputs to the attention layers; there is no implied order or position. The transformer adds the slopes of a sine and cosine wave to the token vectors to encode position.

Attention layers have three inputs: key (k), value(v), and query (q). This layer is self-attention if the query, key, and value are the same. The key and value pairs specify the information that the query operates upon. The attention layer learns what positions of data to focus upon.

The transformer presents the position encoded embedding vectors to the first self-attention segment in the encoder layer. The output from the attention is normalized and ultimately becomes the hidden state after all encoder layers are processed.

The hidden state is only calculated once per query. Once the input Spanish sentence becomes a hidden state, this value is presented repeatedly to the decoder until the decoder forms the final Spanish sentence.

This section presented a high-level introduction to transformers. In the next part, we will implement the encoder and apply it to time series. In the following chapter, we will use [Hugging Face](https://huggingface.co/) transformers to perform natural language processing.





## **LangChain, ChatGPT and NLP**

Large Language Models (LLMs) such as GPT have brought AI into mainstream use. LLMs allow regular users to interact with AI using natural language. Most of these language models require extreme processing capabilities and hardware. Because of this, application programming interfaces (APIs) accessed through the Internet are becoming common entry points for these models. One of the most compelling features of services like ChatGPT is their availability as an API. But before we dive into the depths of coding and integration, let's understand what an API is and its significance in the AI domain.

API stands for Application Programming Interface. Think of it as a bridge or a messenger that allows two different software applications to communicate. In the context of AI and machine learning, APIs often allow developers to access a particular model or service without having to house the model on their local machine. This technique can be beneficial when the model in question, like ChatGPT, is large and resource-intensive.

In the realm of AI, APIs have several distinct advantages:

* Scalability: Since the actual model runs on external servers, developers don't need to worry about scaling infrastructure.
* Maintenance: You get to use the latest and greatest version of the model without constantly updating your local copy.
* Cost-Effective: Leveraging external computational resources can be more cost-effective than maintaining high-end infrastructure locally, especially for sporadic or one-off tasks.
* Ease of Use: Instead of diving into the nitty-gritty details of model implementation and optimization, developers can directly utilize its capabilities with a few lines of code.

In this section, we won't be running the neural network computations locally. Instead, our PyTorch code will communicate with the OpenAI API to access and harness the abilities of ChatGPT. The actual execution of the neural network code happens on OpenAI servers, bringing forth a unique synergy of PyTorch's flexibility and ChatGPT's conversational mastery.

In this section, we will make use of the OpenAI ChatGPT API. Further information on this API can be found here:

* [OpenAI API Login/Registration](https://platform.openai.com/apps)
* [OpenAI API Reference](https://platform.openai.com/docs/introduction/overview)
* [OpenAI Python API Reference](https://platform.openai.com/docs/api-reference/introduction?lang=python)
* [OpenAI Python Library](https://github.com/openai/openai-python)
* [OpenAI Cookbook for Python](https://github.com/openai/openai-cookbook/)
* [LangChain](https://www.langchain.com/)


## **Installing LangChain to use the OpenAI Python Library**

As we delve deeper into the intricacies of deep learning, it's crucial to understand that the tools and platforms we use are as versatile as the concepts themselves. When it comes to accessing ChatGPT, a state-of-the-art conversational AI model developed by OpenAI, there are two predominant pathways:

Direct API Access using Python's HTTP Capabilities: Python, with its rich library ecosystem, provides utilities like requests to directly communicate with APIs over HTTP. This method involves crafting the necessary API calls, handling responses, and error checking, giving the developer a granular control over the process.

Using the Official OpenAI Python Library: OpenAI offers an official Python library, aptly named openai, that simplifies the process of integrating with ChatGPT and other OpenAI services. This library abstracts many of the intricacies and boilerplate steps of direct API access, offering a streamlined and user-friendly approach to interacting with the model.

Each approach has its advantages. Direct API access provides a more hands-on, granular approach, allowing developers to intimately understand the intricacies of each API call. On the other hand, using the openai library can accelerate development, reduce potential errors, and allow for a more straightforward integration, especially for those new to API interactions.

We will make use of the OpenAI API through a library called LangChain. LangChain is a framework designed to simplify the creation of applications using LLMs. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis. LangChain allows you to quickly change between different underlying LLMs with minimal code changes.

The following command installs the **LangChain** library and needed OpenAI LLM connectors.

In [2]:
!pip install langchain langchain_openai > /dev/null

## **Obtaining an OpenAI API Key**

In order to delve into the practical exercises and code demonstrations within this section, students will need to obtain an **OpenAI API key**. This key grants access to OpenAI's services, including the ChatGPT functionality we'll be exploring. It's important to note that there is a nominal cost associated with the usage of this key, depending on the volume and intensity of requests made to OpenAI's servers.

To obtain an OpenAI API key, access this [site](https://platform.openai.com/apps).

In [6]:
# This is the model you will generally use for this class
LLM_MODEL = 'gpt-3.5-turbo-1106'

We begin with a very basic query to LangChain, we ask LangChain what are the 5 largest cities in the USA.


In [3]:
from google.colab import userdata
from langchain_openai import OpenAI, ChatOpenAI

# Retrieve the OpenAI API key and store it in a variable
OPENAI_KEY = userdata.get('OPENAI_KEY')

# Ensure that the API key is correctly set
if not OPENAI_KEY:
    raise ValueError("OpenAI API key is not set. Please check if you have stored the API key in userdata.")

LLM_MODEL = 'gpt-3.5-turbo-1106'

# Initialize the OpenAI LLM (Language Learning Model) with your API key
llm = ChatOpenAI(openai_api_key=OPENAI_KEY, model=LLM_MODEL, temperature=0)

# Define the question
question = "What are the five largest cities in the USA by population?"

# Use Langchain to call the OpenAI API
# The method and parameters might differ based on the Langchain version
response = llm.invoke(question)

# Print the response
print(response.content)


1. New York City, New York
2. Los Angeles, California
3. Chicago, Illinois
4. Houston, Texas
5. Phoenix, Arizona


As you can see, the response from LangChain is in regular English, complete with formatting. While the formatting may make it easier to read, we often have to parse the results given to us by LLMs. Later, we will see that LangChain can help with this as well. You will also notice that we specified a value of 0 for **temperature**; this instructs the LLM to be less creative with its responses and more consistent. Because we are working primarily with data extraction in this section, a low temperature will give us more consistent results.

## Working with Prompts

We will often need to construct complex prompts that incorporate multiple variables into the final prompt. We can use normal Python string handling to achieve this. Lets use ChatGPT to translate from French to English, using normal Python F-Strings to build the prompt.

In [4]:
#
text = """Laissez les bons temps rouler"""
style = "American English"

prompt = f"""Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

response = llm.invoke(prompt)

# Print the response
print(response.content)

"Let the good times roll"


We can use LangChain to help us build dynamic prompts.


In [5]:
from langchain.prompts import ChatPromptTemplate

template_text = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

prompt_template = ChatPromptTemplate.from_template(template_text)



We can now fill in the blanks for this prompt and observe the prompt created, which is a text string.


In [6]:
#
prompt = prompt_template.format_messages(
                    style="American English",
                    text="千里之行，始于足下。")

print(type(prompt))
print(type(prompt[0]))

print(prompt[0])

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>
content='Translate the text that is delimited by triple backticks into a style that is American English. text: ```千里之行，始于足下。```\n' additional_kwargs={} response_metadata={}


This newly constructed prompt can now perform the intended task of translation.

In [7]:
# Call the LLM to translate to the style of the customer message
response = llm.invoke(prompt)
print(response)

content='" A journey of a thousand miles begins with a single step."' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 42, 'total_tokens': 56, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_02b58774e7', 'finish_reason': 'stop', 'logprobs': None} id='run-27e01b8f-28ae-4231-9f7f-c419bf3f7873-0' usage_metadata={'input_tokens': 42, 'output_tokens': 14, 'total_tokens': 56, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


## **Processing Output**

We will now consider a more complex text extraction example and see how LangChain can help us extract multiple values returned by ChatGPT. Here, we will see how three fields can be extracted from a product description.


In [8]:
# Original code

from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

material_schema = ResponseSchema(name="material",
                             description="What is the material that this \
                             item is made of? If unknown, make an estimate.")
description_schema = ResponseSchema(name="shape",
                                      description="What is the shape of this \
                                      item? If unknown, return null.")
who_schema = ResponseSchema(name="who",
                                    description="Who is the likely user of \
                                    this item? If unkown, make an estimate.")

response_schemas = [material_schema,
                    description_schema,
                    who_schema]


As you can see from the above code, we are extracting three fields from the product description: material, shape, and who. We describe LangChain for each to instruct LangChain of what each field is, which helps to find it in the product description. Next we construct a StructuredOutputParser to actually obtain this data.


In [9]:
# Orignal code
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

prompt = ChatPromptTemplate.from_template(template="""
For the product text, extract this information in valid JSON, with commas. \
{format_instructions}
Text:
{text}""")

product_description = """\
Ross Brachiosaurus has a gentle spirit that any dog will quickly love. \
His long body lends itself for tossing or tugging alone or with friends, \
and his size makes him an excellent cuddle companion.
"""


We can now execute these three parts: the prompt builder, invoke the LLM itself, and then finally parse the output from the LLM. This gives us a dictionary containing these three fields.


In [10]:
result1 = prompt.invoke({'text':product_description,'format_instructions':format_instructions})
result2 = llm.invoke(result1)
result3 = output_parser.invoke(result2)
print(result3)


{'material': 'plush', 'shape': 'dinosaur', 'who': 'dog'}


Often you will have multiple components in langchain that you must call in a "chain", to do this you can construct a chain.


In [11]:
chain = prompt | llm | output_parser
chain.invoke({'text':product_description,'format_instructions':format_instructions})


{'material': 'plush', 'shape': 'dinosaur', 'who': 'dog'}

As you can see, the number of successful episodes generally increases as training progresses. It is not advisable to stop the first time we observe 100% success over 1,000 episodes. There is a randomness to most games, so it is not likely that an agent would retain its 100% success rate with a new run. It might be safe to stop training once you observe that the agent has gotten 100% for several update intervals.

## **Application to Text Extraction**

Language model-based learning, commonly abbreviated as LLM, has numerous applications in the world of business. One prevalent utilization of LLM is in the domain of text extraction. Text extraction focuses on the retrieval of specific pieces of information from a larger body of text. For instance, in scenarios where a dataset contains varied information about individuals—ranging from birthdays to job details—one can employ LLM to zero in on just the birthdays, efficiently filtering out extraneous data. The power of LLM lies in its ability to discern context and extract relevant details based on the user's requirements, as showcased in the code that adeptly identifies and extracts birthday details while disregarding other particulars.

In [12]:
from langchain.prompts import ChatPromptTemplate

PROMPT = """
You are to extract any birthdays from the provided text, return the " \
date in the form 10-FEB-1990, or NONE if no birthday.

text: {text}"""

prompt_template = ChatPromptTemplate.from_template(PROMPT)

INPUT = "John was born on June 14, 1995, he was married on May 8, 2015."

chain = prompt_template | llm

result = chain.invoke({'text':INPUT})
print(result.content)


14-JUN-1995


The same code can process a series of text strings. The dates in these strings are in a variety of different formats. The LLM is able to parse and find the needed birthdays and ignore other information. Notice that sometimes the date is not formatted as requested or multiple dates return. Soon we will learn about prompt engineering, which solves some of these problems.


In [13]:

LIST = [
  "Anna started her first job on 15th January 2012. She was born on March 5, 1990.",
  "On 04/14/2007, Michael graduated from college. He was born on 20th July 1985.",
  "Born on 22nd October 1992, Sophia got married on 11.11.2016.",
  "Graduating from high school on June 5, 2005, was a big moment for Lucas. His birth date is 02/17/1987.",
  "Isabelle began her professional journey on 01/09/2016, having been born on December 3, 1994.",
  "Liam was born on May 12, 1988. He celebrated his wedding on 07-15-2014.",
  "Eva celebrated her college graduation on 20-05-2013. Her birthday falls on April 25, 1991.",
  "In 2006, specifically on 03.03.2006, Daniel started his first job. He came into this world on January 8, 1984.",
  "On 05.25.2011, Emily donned her graduation gown. Her birthdate is September 16, 1993.",
  "Henry marked his birthday on 11/30/1989. He tied the knot on October 10, 2017."
]

for item in LIST:
  response = chain.invoke({'text':item})

  print(response.content)


Output: 05-MAR-1990
20-JUL-1985
22-OCT-1992
02/17/1987
December 3, 1994
12-MAY-1988
April 25, 1991
08-JAN-1984
16-SEP-1993
11/30/1989


## **Lesson Turn-in**

When you have completed and run all of the code cells, use the **File --> Print.. --> Save to PDF** to generate a PDF of your Colab notebook. Save your PDF as `Copy of Class_04_1.lastname.pdf` where _lastname_ is your last name, and upload the file to Canvas.

## **Lizard Tail**

## **UNIVAC**

![___](https://upload.wikimedia.org/wikipedia/commons/2/2f/Univac_I_Census_dedication.jpg)

**UNIVAC (Universal Automatic Computer)** was a line of electronic digital stored-program computers starting with the products of the Eckert–Mauchly Computer Corporation. Later the name was applied to a division of the Remington Rand company and successor organizations.

The BINAC, built by the Eckert–Mauchly Computer Corporation, was the first general-purpose computer for commercial use, but it was not a success. The last UNIVAC-badged computer was produced in 1986.

### **History and structure**

**UNIVAC Sperry Rand label**

J. Presper Eckert and John Mauchly built the ENIAC (Electronic Numerical Integrator and Computer) at the University of Pennsylvania's Moore School of Electrical Engineering between 1943 and 1946. A 1946 patent rights dispute with the university led Eckert and Mauchly to depart the Moore School to form the Electronic Control Company, later renamed Eckert–Mauchly Computer Corporation (EMCC), based in Philadelphia, Pennsylvania. That company first built a computer called BINAC (BINary Automatic Computer) for Northrop Aviation (which was little used, or perhaps not at all). Afterwards, the development of UNIVAC began in April 1946.[1] UNIVAC was first intended for the Bureau of the Census, which paid for much of the development, and then was put in production.

With the death of EMCC's chairman and chief financial backer Henry L. Straus in a plane crash on October 25, 1949, EMCC was sold to typewriter, office machine, electric razor, and gun maker Remington Rand on February 15, 1950. Eckert and Mauchly now reported to Leslie Groves, the retired army general who had previously managed building The Pentagon and led the Manhattan Project.

The most famous UNIVAC product was the UNIVAC I mainframe computer of 1951, which became known for predicting the outcome of the U.S. presidential election the following year: this incident is noteworthy because the computer correctly predicted an Eisenhower landslide over Adlai Stevenson, whereas the final Gallup poll had Eisenhower winning the popular vote 51–49 in a close contest.

The prediction led CBS's news boss in New York, Siegfried Mickelson, to believe the computer was in error, and he refused to allow the prediction to be read. Instead, the crew showed some staged theatrics that suggested the computer was not responsive, and announced it was predicting 8–7 odds for an Eisenhower win (the actual prediction was 100–1 in his favour).

When the predictions proved true—Eisenhower defeated Stevenson in a landslide, with UNIVAC coming within 3.5% of his popular vote total and four votes of his Electoral College total—Charles Collingwood, the on-air announcer, announced that they had failed to believe the earlier prediction.

The United States Army requested a UNIVAC computer from Congress in 1951. Colonel Wade Heavey explained to the Senate subcommittee that the national mobilization planning involved multiple industries and agencies: "This is a tremendous calculating process...there are equations that can not be solved by hand or by electrically operated computing machines because they involve millions of relationships that would take a lifetime to figure out." Heavey told the subcommittee it was needed to help with mobilization and other issues similar to the invasion of Normandy that were based on the relationships of various groups.

The UNIVAC was manufactured at Remington Rand's former Eckert-Mauchly Division plant on W Allegheny Avenue in Philadelphia, Pennsylvania. Remington Rand also had an engineering research lab in Norwalk, Connecticut, and later bought Engineering Research Associates (ERA) in St. Paul, Minnesota. In 1953 or 1954 Remington Rand merged their Norwalk tabulating machine division, the ERA "scientific" computer division, and the UNIVAC "business" computer division into a single division under the UNIVAC name. This severely annoyed those who had been with ERA and with the Norwalk laboratory.

In 1955 Remington Rand merged with Sperry Corporation to become Sperry Rand. General Douglas MacArthur, then the chairman of the Board of Directors of Remington Rand, was chosen to continue in that role in the new company. Harry Franklin Vickers, then the President of Sperry Corporation, continued as president and CEO of Sperry Rand. The UNIVAC division of Remington Rand was renamed the Remington Rand Univac division of Sperry Rand. William Norris was put in charge as Vice-President and General Manager reporting to the President of the Remington Rand Division (of Sperry Rand).